In [114]:
#IMPORTS

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import requests

from bs4 import BeautifulSoup
import time

In [115]:
df = pd.DataFrame(columns=['href_list']) # creates master dataframe 

# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

for listings in range(1,2):
    page_num = str(listings)
    url = 'https://www.barnesandnoble.com/b/books/_/N-1z13w9sZ1sZ29Z8q8?Nrpp=20&page=' + page_num
    driver.get(url)
    # freeze/halt python so url loads completely
    driver.implicitly_wait(60)
    
    #find hrefs
    div_elements = driver.find_elements(By.CSS_SELECTOR, "div.product-shelf-title.product-info-title.pt-xs")
    
    href_list = []
    for div_element in div_elements:
        elements = div_element.find_elements(By.TAG_NAME, "a")
        for element in elements:
            href = element.get_attribute('href')
            href_list.append(href)

    urls_df = pd.DataFrame(href_list, columns=['href_list']) # creates master dataframe
    
#driver.close()


/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_4263/3521870330.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [116]:
# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

book_details = []

for index, href in enumerate(href_list):
    driver.get(href)
    driver.implicitly_wait(60)

    # find titles
    titles = driver.find_elements(By.TAG_NAME, 'h1')
    title_list = []
    for t in range(len(titles)):
        title_list.append(titles[t].text)

    #find blurbs
    parent_div = None
    try:
        parent_div = driver.find_element(By.CLASS_NAME, "book-seller-cntnt")
    except NoSuchElementException:
        try:
            parent_div = driver.find_element(By.CLASS_NAME, "overview-cntnt ov-cont-tab")
        except NoSuchElementException:
            pass

        
    
    blurb_list = []
    if parent_div:
        for p in parent_div.find_elements(By.TAG_NAME, "p"):
            blurb_list.append(p.text)
            for b in p.find_elements(By.TAG_NAME, "b"):
                blurb_list.append(b.text)

    if not title_list:
        continue
    
    book = [href, title_list[1] if len(title_list) > 1 else title_list[0], blurb_list[0] if blurb_list else np.nan]
    book_details.append(book)

driver.quit()

titles_df = pd.DataFrame(book_details, columns=['href', 'title', 'blurb'])



/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_4263/3859491542.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [117]:
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   href    20 non-null     object
 1   title   20 non-null     object
 2   blurb   5 non-null      object
dtypes: object(3)
memory usage: 608.0+ bytes


In [109]:
titles_df

,href,title,blurb
0,https://www.barnesandnoble.com/w/generaci-n-id...,Generación idiota: Una crítica al adolescentrismo,El controversial escritor Agustín Laje adviert...
1,https://www.barnesandnoble.com/w/spare-prince-...,Spare: En la sombra,La controversial autobiografía del príncipe Ha...
2,https://www.barnesandnoble.com/w/volver-a-empe...,Volver a empezar / It Starts with Us (Spanish ...,¡Colleen Hoover nos brinda su magia nuevamente...
3,https://www.barnesandnoble.com/w/v-monos-a-la-...,¡Vámonos a la Estufa! con Janet Jauja Cocina M...,"Jauja, la popular chef estrella de YouTube, no..."
4,https://www.barnesandnoble.com/w/ruge-o-espera...,RUGE: O espera a ser devorado (Spanish Edition),Con su clásico agresivo pero muy alentador est...
5,https://www.barnesandnoble.com/w/donde-crece-e...,Donde crece el helecho rojo / Where the Red Fe...,NaN
6,https://www.barnesandnoble.com/w/qui-n-es-mich...,¿Quién es Michael Jordan?,NaN
7,https://www.barnesandnoble.com/w/qui-n-es-soni...,¿Quién es Sonia Sotomayor?,NaN
8,https://www.barnesandnoble.com/w/el-reino-de-l...,"El reino de los malditos, Vol. 2 / Kingdom of ...",NaN
9,https://www.barnesandnoble.com/w/despu-s-de-di...,Después de diciembre / After December,NaN
